In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Tu API key de Alpha Vantage
api_key = 'YOUR_ALPHA_VANTAGE_API_KEY'
ticker = 'AAPL'

In [ ]:
# Balance Sheet
def get_balance_sheet(ticker):
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    return pd.DataFrame(data['annualReports'])

# Income Statement
def get_income_statement(ticker):
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    return pd.DataFrame(data['annualReports'])

# Cash Flow
def get_cash_flow(ticker):
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    return pd.DataFrame(data['annualReports'])

In [ ]:
def preparar_datos_avanzado(ticker):
    bs = get_balance_sheet(ticker)
    is_ = get_income_statement(ticker)
    cf = get_cash_flow(ticker)

    for df in [bs, is_, cf]:
        df.set_index('fiscalDateEnding', inplace=True)
        df.sort_index(inplace=True)
        for col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    ni = is_['netIncome']
    revenue = is_['totalRevenue']
    cfo = cf['operatingCashflow']
    capex = cf['capitalExpenditures']
    total_assets = bs['totalAssets']

    accruals = ni - cfo
    delta_accruals = accruals.diff()
    avg_assets = (total_assets + total_assets.shift(1)) / 2
    sloan_ratio = delta_accruals / avg_assets
    ccr = cfo / ni
    capex_cfo = capex / cfo
    tata = (ni - cfo) / total_assets
    m_score = -4.84 + 4.679 * tata

    df = pd.DataFrame({
        'Revenue': revenue,
        'Net Income': ni,
        'CFO': cfo,
        'CapEx': capex,
        'Sloan Ratio': sloan_ratio,
        'CCR': ccr,
        'CapEx/CFO': capex_cfo,
        'Beneish M-Score': m_score
    })

    return df.dropna()

In [ ]:
def plot_calidad_ganancias(df):
    df[['Growth NI (%)', 'Growth CFO (%)']].plot(kind='bar', figsize=(10,6), title="Crecimiento CFO vs Net Income")
    plt.axhline(0, color='black', linewidth=1)
    plt.tight_layout()
    plt.show()

    df['Sloan Ratio'].plot(kind='bar', title="Sloan Ratio (Accruals/Assets)", figsize=(10,6))
    plt.axhline(0, color='black', linewidth=1)
    plt.tight_layout()
    plt.show()

In [ ]:
def calcular_ccr(df):
    df['CCR'] = df['CFO'] / df['Net Income']
    return df

In [ ]:
def calcular_beneish(df, bs, is_):
    # Requiere ventas y activos totales para TATA
    total_assets = bs['totalAssets']
    ni = is_['netIncome']
    cfo = df['CFO']

    # TATA = (NI - CFO) / Total Assets
    tata = (ni - cfo) / total_assets

    # Solo una versión simplificada del M-Score usando TATA
    m_score = -4.84 + 4.679 * tata.fillna(0)

    df['TATA'] = tata
    df['Beneish M-Score'] = m_score
    return df

In [ ]:
def color_semaforo(val, metric):
    
    if metric == 'Sloan Ratio':
        if val > 0.15:
            return 'background-color: red'
        elif val > 0.10:
            return 'background-color: yellow'
        else:
            return 'background-color: lightgreen'

    elif metric == 'CCR':
        if val < 0.8:
            return 'background-color: red'
        elif val < 1.0:
            return 'background-color: yellow'
        else:
            return 'background-color: lightgreen'

    elif metric == 'Beneish M-Score':
        if val > -1.78:
            return 'background-color: red'
        elif val > -2.0:
            return 'background-color: yellow'
        else:
            return 'background-color: lightgreen'

    return ''

In [ ]:
def aplicar_semaforo(df):
    return df.style.applymap(lambda x: color_semaforo(x, 'Sloan Ratio'), subset=['Sloan Ratio'])\
                   .applymap(lambda x: color_semaforo(x, 'CCR'), subset=['CCR'])\
                   .applymap(lambda x: color_semaforo(x, 'Beneish M-Score'), subset=['Beneish M-Score'])

In [ ]:
# Preparar datos
df_avanzado = preparar_datos_avanzado(ticker)

# Obtener datos originales para Beneish
bs = get_balance_sheet(ticker)
is_ = get_income_statement(ticker)

# Calcular Beneish
df_completo = calcular_beneish(df_avanzado, bs, is_)

# Ver resultados
df_alertas = df_completo[['Sloan Ratio', 'CCR', 'Beneish M-Score']]
aplicar_semaforo(df_alertas)

In [ ]:
df_completo[['Sloan Ratio', 'CCR', 'CapEx/CFO']].plot(kind='barh', figsize=(10,6), title=f"Indicadores clave - {ticker}")
plt.grid(True)

In [ ]:
def diagnostico_ganancias(df, ticker):
    year = df.index[-1]
    sloan = df.loc[year, 'Sloan Ratio']
    ccr = df.loc[year, 'CCR']
    mscore = df.loc[year, 'Beneish M-Score']

    resumen = f"""Diagnóstico Financiero para {ticker} - {year}
▪ Sloan Ratio: {sloan:.2f} {'!!' if sloan > 0.15 else '[Bueno]'}
▪ Cash Conversion Ratio: {ccr:.2f} {'!!' if ccr < 0.8 else '[Bueno]'}
▪ Beneish M-Score: {mscore:.2f} {'!!' if mscore > -1.78 else '[Bueno]'}
"""
    print(resumen)

In [ ]:
diagnostico_ganancias(df_completo, ticker)

In [ ]:
import matplotlib.pyplot as plt

def graficar_tendencias(df, ticker):
    plt.figure(figsize=(12,6))
    df[['Sloan Ratio', 'CCR', 'CapEx/CFO']].plot(marker='o', title=f"Calidad de Ganancias - {ticker}", figsize=(12,6))
    plt.axhline(0.1, color='gray', linestyle='--', label="Límite Sloan sano (0.10)")
    plt.ylabel("Valor")
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    df['Beneish M-Score'].plot(marker='o', figsize=(12,5), title=f"Beneish M-Score - {ticker}")
    plt.axhline(-1.78, color='red', linestyle='--', label="Umbral de alerta")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def diagnostico_tendencia(df, ticker):
    ultimos = df.tail(3)
    print(f"Tendencia de {ticker} (últimos 3 años)")
    print(f"▪ Sloan Ratio: ({ultimos['Sloan Ratio'].iloc[0]:.2f} -> {ultimos['Sloan Ratio'].iloc[-1]:.2f})")
    print(f"▪ CCR: ({ultimos['CCR'].iloc[0]:.2f} -> {ultimos['CCR'].iloc[-1]:.2f})")

In [ ]:
df_multi = preparar_datos_avanzado(ticker)
graficar_tendencias(df_multi, ticker)
diagnostico_tendencia(df_multi, ticker)